In [12]:
from pydantic import BaseModel, ConfigDict, BeforeValidator, Field, EmailStr
from enum import Enum
from typing import Optional, Annotated

PyObjectId = Annotated[str, BeforeValidator(str)]

class SeatType(Enum):
    REGULAR = 0
    PREMIUM = 1
    VIP = 2

class SeatStatus(Enum):
    AVAILABLE = 0
    BOOKED = 1
    RESERVED = 2


class Seat(BaseModel):
    id: Optional[PyObjectId] = Field(alias='_id', default=None) # primary key
    uid: str
    concert_id: str
    venue: str
    seat_number: str
    seat_type: SeatType
    price: float
    status: SeatStatus

    model_config = ConfigDict(use_enum_values=True)


In [13]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

# get password and uri for database connection
load_dotenv()
db_password = os.getenv('MONGO_PASSWORD')
uri = os.getenv('DATABASE_URI').replace('db_password', db_password)

# create client and connect to server
client = MongoClient(uri, server_api=ServerApi('1'))

# ping for confirmation and update databases
db = client['bigcat']


In [14]:
import json

with open('seating_chart.json', 'r') as file:
    data = json.load(file)

del data['schema']
for key, val in data.items():
    print(key, val)

3TEN Austin City Limits Live {'layout': {'GA': [1, 0, 350, 50]}}
Antone's Austin {'layout': {'GA': [1, 0, 750, 15]}}
Austin City Limits Live at The Moody Theater {'layout': {'GA': [1, 0, 300, 35, 50], 'MEZZ': [8, 3, 25, 45], 'BALC': [6, 12, 22, 30]}}
Brauntex Performing Arts Theatre {'layout': {'GA': [1, 0, 586, 40]}}
Cheatham Street Warehouse {'layout': {'GA': [1, 0, 500, 20]}}
Empire - Control Room {'layout': {'GA': [1, 0, 300, 35]}}
Germania Insurance Amphitheater {'layout': {'GA-Pit': [1, 0, 800, 50], '100-Level': [7, 12, 30, 65], '200-Level': [4, 31, 30, 55], 'GA-Lawn': [1, 0, 7000, 20]}}
Gruene Hall {'layout': {'GA': [1, 0, 800, 20]}}
Long Center for the Performing Arts - Dell Hall {'layout': {'Orchestra': [3, 17, 20, 45], 'Parterre': [3, 9, 25, 25]}}
Moody Amphitheater {'layout': {'GA': [1, 0, 5000, 40]}}
Moody Center {'layout': {'Pit': [2, 0, 200, 200], 'GA': [9, 0, 100, 100], '100-Level': [14, 10, 30, 90], '200-Level': [9, 7, 30, 70], 'S-Level': [9, 3, 7, 105], 'P-Level': [9, 

In [ ]:
from uuid import uuid4

for venue, info in data.items():
    docs = db['concert'].find({'venue': venue})

    for doc in docs:
        concert_id = doc['uid']
        venue = key
        
        for section, section_data in info.items():
            num_sections = section_data[0]
            num_rows = section_data[1]
            num_seats = section_data[2]
            price_min = section_data[3]

            # loop for seat number. maybe zip num_sections, num_rows, and num_seats

            seat = Seat(
                uid=str(uuid4()),
                concert_id=concert_id,
                venue=venue,
                seat_number=f'{}{}{}',
                seat_type=SeatType.REGULAR,
                price=price_min,
                status=SeatStatus.AVAILABLE
            )

# for concert in concert_data:
#     db['concert'].insert_one(concert.__dict__)

{'_id': ObjectId('68c41e5fe87102fa1497f92e'), 'id': None, 'uid': '5df2c597-1a9a-46e6-a0da-2f0d0a7a525c', 'artist': 'Aloe Blacc', 'tour_name': None, 'venue': '3TEN Austin City Limits Live', 'location': 'Austin, TX, USA', 'concert_datetime': datetime.datetime(2025, 9, 12, 21, 0), 'status': 1, 'update_datetime': datetime.datetime(2025, 9, 12, 8, 20, 50, 525000)}
{'_id': ObjectId('68c41e5ee87102fa1497f90e'), 'id': None, 'uid': '6e70039e-b9ce-4e74-b518-80c9080cd34a', 'artist': 'Addison Rae', 'tour_name': None, 'venue': 'Austin City Limits Live at The Moody Theater', 'location': 'Austin, TX, USA', 'concert_datetime': datetime.datetime(2025, 9, 22, 20, 0), 'status': 1, 'update_datetime': datetime.datetime(2025, 9, 12, 8, 20, 50, 525000)}
{'_id': ObjectId('68c41e5fe87102fa1497f930'), 'id': None, 'uid': '506d6ad0-a742-4356-af03-573e99305f85', 'artist': 'Donny Edwards', 'tour_name': None, 'venue': 'Brauntex Performing Arts Theatre', 'location': 'Austin, TX, USA', 'concert_datetime': datetime.dat

In [ ]:
from pydantic import BaseModel, BeforeValidator, ConfigDict
from enum import Enum
from datetime import datetime
from typing import Optional, Annotated
from uuid import uuid4
import requests

PyObjectId = Annotated[str, BeforeValidator(str)]

class BookingStatus(Enum):
    PENDING = 0
    CONFIRMED = 1
    CANCELED = 2

class PaymentStatus(Enum):
    PENDING = 0
    SUCCESS = 1
    FAILED = 2
    CANCELED = 3
    REFUNDED = 4
    AUTHORIZED = 5


class Booking(BaseModel):
    uid: str
    user_id: str  # foreign key
    concert_id: str # foreign key
    venue: str
    seats: list[str] # foreign keys when exploded
    total_price: float
    payment_status: PaymentStatus
    status: BookingStatus
    request_datetime: datetime
    update_datetime: datetime
    confirmation_id: str

    model_config = ConfigDict(use_enum_values=True)
    

booking = Booking(
    uid=str(uuid4()),
    user_id='7417d724-83b4-4d5c-b00f-673ffc9fbf9e',
    concert_id='6e70039e-b9ce-4e74-b518-80c9080cd34a',
    venue='Austin City Limits Live at The Moody Theater',
    seats=[],
    total_price=[],
    status=BookingStatus.CONFIRMED,
    request_datetime=datetime.now(),
    update_datetime=datetime.now(),
    confirmation_id=str(uuid4())[:8]
)


response = requests.post("http://127.0.0.1:8000/auth/", json=register_data.model_dump())
response

<Response [201]>